In [1]:
import numpy as np
import tensorflow as tf

%load_ext autoreload
%autoreload 2

/Users/Ikkei/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
with open('data/profile_intro.csv') as f:
    data = f.read().splitlines()

text = ''.join(data)

chars = sorted(set(text))
char_to_ind = {c: i for i, c in enumerate(chars)}
ind_to_char = {v: k for k, v in char_to_ind.items()}

text_as_int = np.array([char_to_ind[c] for c in text])

In [3]:
def generate_batches(array, batch_shape):
    num_samples, num_chars = batch_shape
    total_items = num_samples * num_chars
    num_batches = int(len(array) / total_items)

    # Keep the full batches and ignore the left.
    # from (1115394,) to (1115300,)
    array = array[:total_items*num_batches]

    # from (1115300,) to (10, 111530)
    array = array.reshape((num_samples, -1))

    i = 0
    while True:
        x = array[:, i:i+num_chars]
        y = np.roll(x, shift=-1, axis=1)
        
        if i >= (array.shape[1] - num_chars):
            i = 0
        else:
            i += num_chars
            
        yield x, y

In [4]:
from char_rnn import CharRNN


batch_shape = (100, 100)
batches = generate_batches(text_as_int, batch_shape)
model = CharRNN(char_to_ind, batch_shape, rnn_size=256, num_layers=3, learning_rate=0.005, grad_clip=5)
model.train(batches, iters=200, keep_prob=0.5)

step: 0 loss: 4.5517


In [6]:
samp = model.predict(prime="Hi, I'm ", num_char=100)
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i0_l256_ckpt
Hi, I'm eiiseatae  ee e   iea taet    a a eaeatt eat  e        i  i i     e ia t  ti a     te  aa i a     ie
